In [ ]:
#PART 1 : LOAD ATTENDANCE DATA EXPORT EXCEL CONVERTED TO CSV NAMED AS data.csv
import pandas as pd
from IPython.display import display
import requests

# creates a dataframe with the csv
df = pd.read_csv('data.csv')

# 2) Locate the row with "SY Master" and set the start index to the very next row
keyword = 'SY Master'
first_idx = None
for idx, row in df.iterrows():
    # check every cell in the row for the keyword
    if row.astype(str).str.contains(keyword).any():
        first_idx = idx
        break
if first_idx is None:
    raise ValueError("Keyword 'SY Master' not found in the CSV") #error handling
start_idx = first_idx + 1

# 3) The five columns where you mark Present(P) and Absent(A)
columns_to_count = ['Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5']

# 4) The column holding the student’s name (first column)
name_column = df.columns[0]

# 5) Loop through only the rows after SY Master, tally P's and A's
results = []
for i in range(start_idx, len(df)):
    row = df.iloc[i]
    row_name = row[name_column]

    present_count = 0
    absent_count = 0
    for col in columns_to_count:
        if col in df.columns:
            cell = str(row[col]).upper() if pd.notna(row[col]) else ''
            present_count += cell.count('P')
            absent_count  += cell.count('A')

    results.append({
        'Name': row_name,
        'P': present_count,
        'A': absent_count
    })

# 6) Turn your results into a DataFrame and display
results_df = pd.DataFrame(results)
print("✅ Attendance tally after 'SY Master':")
display(results_df)


In [ ]:
#CONVERT ATTENDANCE BY PARTICIPANT TO A CSV USING CHATGPT THEN NAME pt2.csv
import pandas as pd
from IPython.display import display

# 1) Loads the csv that shows Sy Master and PD hours and shows how many days a student has attended.
results_df = pd.read_csv('pt2.csv')

# 2) Rename columns for convenience
results_df = results_df.rename(columns={
    'Participant Name':           'Name',
    'SY Master Days Attended':    'Days',
    'SY Master Hours':            'Hours'
})

# 3) Build the merge key from "Last, First"
def make_merge_name_from_results(name):
    last, first = [s.strip() for s in name.split(',', 1)]
    return f"{first} {last}".lower()

results_df['merge_name'] = results_df['Name'].apply(make_merge_name_from_results)

# 4) Load emails.csv FOR STUDENT EMAILS so it can merge and generated output will have email for contacting them to meet attendance requirements.
emails_df = pd.read_csv(
    'emails.csv',
    sep='\t',
    header=None,
    names=['Email','AKA','First','Last'],
    engine='python'
)

# 5) Clean up and build its merge key
emails_df['Last']  = emails_df['Last'].str.replace(r'\*$', '', regex=True).str.strip()
emails_df['First'] = emails_df['First'].str.strip()
emails_df['merge_name'] = (emails_df['First'] + ' ' + emails_df['Last']).str.lower()

# 6) Merge Email in
merged = results_df.merge(
    emails_df[['merge_name','Email']],
    on='merge_name',
    how='left'
)

# 7) Split out 0-day and 1-day attendees for contacting students to meet the min requirements per week which is 3 or more days.
zero_days_df = (
    merged.loc[merged['Days'] == 0, ['Name','Email']]
    .reset_index(drop=True)
)

one_day_df = (
    merged.loc[merged['Days'] == 1, ['Name','Email','Hours']]
    .reset_index(drop=True)
)

# 8) Display results
print("👉 SY Master days = 0:")
display(zero_days_df)

print("\n👉 SY Master days = 1:")
display(one_day_df)
